In [1]:
import pandas as pd
import numpy as np

import openpyxl
import os

In [2]:
import pandas as pd
import holoviews as hv
from holoviews import opts
from bokeh.models import HoverTool
import streamlit as st

ImportError: cannot import name 'ImportedStyleSheet' from 'bokeh.models' (/Users/krishnasai-narayanan/anaconda3/lib/python3.11/site-packages/bokeh/models/__init__.py)

In [ ]:
os.getcwd()

In [ ]:
os.chdir('..')

In [ ]:
path = '/Users/krishnasai-narayanan/Desktop/Home/Rutgers - Coursework/Fall 2024/Big Data Algo/big-data/group project/project_data'

In [ ]:
filename = path + '/2019-29/occupation.xlsx'

In [ ]:
occu_obj = openpyxl.load_workbook(path + '/2019-29/occupation.xlsx')

In [ ]:
index_sheet = pd.read_excel(filename)

In [ ]:
index_sheet

In [ ]:
sheet_data = {}

In [ ]:
def basic_data_cleaning(data1):

    data1 = data1.rename(columns={data1.columns[0]: "Job Title"})
    
    try:
        index = data1[data1['Job Title'].str.contains('Footnotes')].index[0]
    except:
        index = data1[data1['Job Title'].str.contains('U.S. Bureau of Labor Statistics')].index[0]
        
    data1 = data1.iloc[:index]

    try:
        data1 = data1.drop('2019 National Employment Matrix code', axis=1)
    except:
        pass
    
    return data1

In [ ]:
for i in index_sheet.index:
    data = pd.read_excel(filename, sheet_name=i+1, skiprows=1)
    
    print(index_sheet.loc[i].iloc[0])
    
    sheet_data[index_sheet.loc[i].iloc[0]] = basic_data_cleaning(data)

## Data Visualizations on Individual Sheets of Data

### Table 1.1 Employment by major occupational group, 2019 and projected 2029

In [ ]:
# Enable Holoviews Bokeh backend
hv.extension('bokeh')

df = pd.DataFrame(sheet_data[index_sheet.loc[0].iloc[0]])

In [ ]:
df.columns

In [ ]:
# Melt the data to a long format for easier plotting
melted_df = df.melt(
    id_vars='Job Title',
    value_vars=['Employment, 2019', 'Employment, 2029'],
    var_name='Year',
    value_name='Employment'
)

In [ ]:
melted_df['Year'] = melted_df['Year'].str.extract(r'(\d+)').astype(int)

In [ ]:
selected_groups = st.sidebar.multiselect(
    "Select Occupational Groups",
    options=df['Job Title'].unique(),
    default=df['Job Title'].unique()
)

filtered_data = melted_df[melted_df['Job Title'].isin(selected_groups)]

plot = hv.Curve(
    filtered_data, 
    kdims=['Year'], 
    vdims=['Employment', 'Job Title']
).groupby('Job Title').overlay()

hover = HoverTool(
    tooltips=[
        ("Group", "@{Job Title}"),
        ("Year", "@{Year}"),
        ("Employment", "@{Employment}")
    ]
)
plot = plot.opts(
    opts.Overlay(
        show_legend=True, 
        legend_position='top_left'
    ),
    opts.Curve(
        tools=[hover], 
        line_width=2, 
        color=hv.Cycle('Category20'), 
        responsive=True, 
        height=400, 
        width=700
    )
)

st.title("Employment Trends (2019-2029)")
st.write("Interactive line plot of employment trends for selected occupational groups.")

In [ ]:
st.bokeh_chart(hv.render(plot, backend='bokeh'))